# Chapter 48: CI/CD for AI/ML - MLOps Pipelines
Automate training, validation, and deployment of AI models.


In [ ]:
import json, numpy as np
from datetime import datetime
print("MLOps Pipelines ready")


In [ ]:
class Pipeline:
    def __init__(self): self.stages=[]
    def add(self,name,fn): self.stages.append((name,fn))
    def run(self,data):
        for name,fn in self.stages:
            try: data=fn(data); print(f"  [OK] {name}")
            except Exception as e: print(f"  [FAIL] {name}: {e}"); return None
        return data
np.random.seed(42)
pipe=Pipeline()
pipe.add("validate",lambda d: d if d else (_ for _ in ()).throw(ValueError("empty")))
pipe.add("featurize",lambda d: [{"features":list(range(8)),"label":x%3} for x in range(len(d))])
pipe.add("train",lambda d: {"accuracy":round(0.85+np.random.rand()*0.1,3),"n":len(d)})
result=pipe.run(list(range(100)))
print("Result:",result)


In [ ]:
class Registry:
    def __init__(self): self.models={}
    def register(self,name,ver,metrics): k=f"{name}@{ver}"; self.models[k]={"name":name,"version":ver,"metrics":metrics,"stage":"staging","ts":datetime.utcnow().isoformat()}; return k
    def promote(self,k): self.models[k]["stage"]="production"
    def get_prod(self,name): return [v for v in self.models.values() if v["name"]==name and v["stage"]=="production"]
reg=Registry()
k=reg.register("threat-clf","v1.2.0",{"accuracy":0.923,"f1":0.911})
reg.promote(k)
print(f"Registered: {k}")
prod=reg.get_prod("threat-clf")
print(f"Production: {[p[chr(118)+chr(101)+chr(114)+chr(115)+chr(105)+chr(111)+chr(110)] for p in prod]}")
baseline={"pkt_size":{"mean":512.0,"std":128.0}}
current={"pkt_size":{"mean":895.0,"std":210.0}}
delta=abs(baseline["pkt_size"]["mean"]-current["pkt_size"]["mean"])
print(f"Data drift detected: {delta/baseline[chr(112)+chr(107)+chr(116)+chr(95)+chr(115)+chr(105)+chr(122)+chr(101)][chr(115)+chr(116)+chr(100)]>1}")
